# SemiBoosted Analysis with BDT

In [1]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import vector
from sklearn.metrics import auc, roc_curve
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

import HH4b.utils as utils

In [2]:
def make_vector(events: pd.DataFrame, obj: str):
    """Create a ``vector`` object from the columns of the dataframe"""
    mstring = "PNetMass" if obj == "ak8FatJet" else "Mass"

    return vector.array(
        {
            "pt": events[f"{obj}Pt"],
            "phi": events[f"{obj}Phi"],
            "eta": events[f"{obj}Eta"],
            "M": events[f"{obj}{mstring}"],
        }
    )

## Load Dataset

In [3]:
year = "2018"  #
dir_name = "23Nov18_WSel_v9_private"
path_to_dir = f"/Users/billyli/UCSD/{dir_name}/"

In [ ]:
# Load your dataset
samples = {
    "hh4b": ["GluGlutoHHto4B_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # cHHH = kappa lambda
    # "hh4b-c2p45": ["GluGlutoHHto4B_cHHH2p45_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # "hh4b-c5": ["GluGlutoHHto4B_cHHH5_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # "hh4b-c0": ["GluGlutoHHto4B_cHHH0_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    "qcd": [
        "QCD_HT-200to300-13TeV",
        "QCD_HT-300to500-13TeV",
        "QCD_HT-500to700-13TeV",
        "QCD_HT-700to1000-13TeV",
        "QCD_HT-1000to1500-13TeV",
        "QCD_HT-1500to2000-13TeV",
        "QCD_HT-2000toInf-13TeV",
    ],
    "ttbar": [
        "TTTo2L2Nu_13TeV",
        "TTToHadronic_13TeV",
        "TTToSemiLeptonic_13TeV",
    ],
}

dirs = {path_to_dir: samples}

filters = [
    [
        # one good fatjet (fatjet with index 0 has the largest Xbb score)
        ("('ak8FatJetPt', '0')", ">=", 300),
        ("('ak8FatJetMsd', '0')", ">=", 60),
        ("('ak8FatJetPNetXbb', '0')", ">=", 0.9734),  # medium WP
        # second fatjet has lower
        ("('ak8FatJetPNetXbb', '1')", "<", 0.9880),  # tight WP
    ]
]

# columns to load
# the parquet files are too big so we can only load a few columns at a time without consumming much memory
load_columns = [
    ("weight", 1),
    ("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
    ("ak8FatJetPNetXbb", 2),
]
# reformat into ("column name", "idx") format for reading multiindex columns
columns = []
for key, num_columns in load_columns:
    for i in range(num_columns):
        columns.append(f"('{key}', '{i}')")


events_dict = {}
for input_dir, samples in dirs.items():
    events_dict = {
        **events_dict,
        # this function will load files (only the columns selected), apply filters and compute a weight per event
        **utils.load_samples(
            input_dir, samples, year, filters=filters, load_weight_noxsec=False, reorder_txbb=False
        ),
    }

samples_loaded = list(events_dict.keys())
keys_loaded = list(events_dict[samples_loaded[0]].keys())
print("Keys in events_dict")
for i in keys_loaded:
    print(i)

## Event cuts

In [5]:
# Higgs candidate selection example
events_raw = pd.concat(
    [events_dict["hh4b"], events_dict["qcd"], events_dict["ttbar"]], keys=["hh4b", "qcd", "ttbar"]
)

In [6]:
# AK4OutsideJet pt cut
jets_outside_raw = make_vector(events_raw, "ak4JetOutside")
j3_raw = jets_outside_raw[:, 0]
j4_raw = jets_outside_raw[:, 1]
j3j4_pt_cut = (j3_raw.pt > 20) & (j4_raw.pt > 20)

In [7]:
combined_filter = j3j4_pt_cut
events = events_raw[combined_filter]

In [ ]:
events.loc["ttbar"]["weight"]

In [ ]:
list(events.keys())

## Define Targets with weights

In [9]:
def get_yw_from_events(events):
    # Define Targets
    events["target"] = 0  # default is background
    events.loc["hh4b", "target"] = 1  # Set to 1 for 'hh4b' samples (signal)

    # define target and weights
    targets_weights = events[["target", "weight"]]
    targets_weights.loc[:, "weight"] = np.abs(targets_weights.loc[:, "weight"].to_numpy())

    # calculate the sum of weights in signal/bg categories
    signal_mask = targets_weights["target"] == 1
    total_signal_weight = targets_weights[signal_mask]["weight"].sum()
    total_bg_weight = targets_weights[~signal_mask]["weight"].sum()

    # balancing classes
    print("total signal weight", total_signal_weight)
    print("total bg weight", total_bg_weight)
    targets_weights.loc[signal_mask, "bal_weight"] = (
        targets_weights.loc[signal_mask, "weight"] / total_signal_weight
    ).to_numpy()
    targets_weights.loc[~signal_mask, "bal_weight"] = (
        targets_weights.loc[~signal_mask, "weight"] / total_bg_weight
    ).to_numpy()

    # normalize weights
    targets_weights.loc[:, "bal_weight"] = (
        targets_weights.loc[:, "bal_weight"] / targets_weights.loc[:, "bal_weight"].max()
    ).to_numpy()

    print("after reweighting and normalizing")
    total_signal_weight = targets_weights[signal_mask]["bal_weight"].sum()
    total_bg_weight = targets_weights[~signal_mask]["bal_weight"].sum()
    print("total signal weight", total_signal_weight)
    print("total bg weight", total_bg_weight)

    return targets_weights

In [ ]:
targets_weights = get_yw_from_events(events)
targets_weights

## Define Features

In [16]:
def get_x_from_events(events):
    multiIndex = events.index
    events = events.reset_index()

    # Use pandas df as feature container
    pdf = pd.DataFrame()

    # H1 features
    fatjets = make_vector(events, "ak8FatJet")  # fatjets sorted by xbb
    h1 = fatjets[:, 0]
    pdf["logPtH1"] = np.log(h1.pt)  #
    pdf["hb_eta"] = h1.eta  #

    # H2 features
    jets_outside = make_vector(events, "ak4JetOutside")  # sorted by b-score
    j3 = jets_outside[:, 0]
    j4 = jets_outside[:, 1]
    h2 = j3 + j4
    pdf["hr_pt"] = h2.pt  #
    pdf["hr_eta"] = h2.eta  #
    pdf["hr_mass"] = h2.mass  #

    pn_b = events.ak4JetOutsidebtagPNetProbb + events.ak4JetOutsidebtagPNetProbbb
    pn_c = events.ak4JetOutsidebtagPNetProbc + events.ak4JetOutsidebtagPNetProbcc
    pn_uds = events.ak4JetOutsidebtagPNetProbuds
    pn_g = events.ak4JetOutsidebtagPNetProbg
    ak4JetOutsidePNetFlavB = pn_b / (pn_b + pn_c + pn_uds + pn_g)
    j3_PNetFlavB = ak4JetOutsidePNetFlavB[0]
    j4_PNetFlavB = ak4JetOutsidePNetFlavB[1]

    j3_btag = j3_PNetFlavB
    j4_btag = j4_PNetFlavB

    pdf["logPtJ3"] = np.log(j3.pt)  #
    pdf["logPtJ4"] = np.log(j4.pt)  #
    pdf["j3_btag"] = j3_btag  #
    pdf["j4_btag"] = j4_btag  #

    pdf["dPhiJ3J4"] = j3.deltaphi(j4)
    pdf["dRJ3J4"] = j3.deltaR(j4)  #
    pdf["j3_j4_ratio"] = j3.pt / j4.pt  #

    # HH features
    hh = h1 + h2
    drh1h2 = np.sqrt((h1.eta - h2.eta) ** 2 + (h1.phi - h2.phi) ** 2)

    pdf["logPtHH"] = np.log(hh.pt)  #
    pdf["HHMass"] = hh.M  #
    pdf["HHEta"] = hh.eta  #  #

    pdf["H1Pt_H2Pt"] = h1.pt / h2.pt  #
    pdf["dphi"] = h1.deltaphi(h2)  #
    pdf["deta"] = h1.deltaeta(h2)  #
    pdf["dRH1H2"] = drh1h2  #

    # Second fatjet
    fj2 = fatjets[:, 1]
    pdf["fj2_pt"] = fj2.pt  #
    pdf["fj2_eta"] = fj2.eta  #
    pdf["fj2_mass"] = fj2.M  #

    features = pdf.set_index(multiIndex)
    return features

In [17]:
features = get_x_from_events(events)

In [18]:
# Splitting the dataset into (training & validating) and testing sets
# train:val:test = 6:2:2
X_train_val, X_test, yw_train_val, yw_test = train_test_split(
    features, targets_weights, test_size=0.2, random_state=42
)

y_test = yw_test["target"]
w_test = yw_test["bal_weight"]

In [19]:
X_train, X_val, yw_train, yw_val = train_test_split(
    X_train_val, yw_train_val, test_size=0.25, random_state=42
)

y_train = yw_train["target"]
w_train = yw_train["bal_weight"]

y_val = yw_val["target"]
w_val = yw_val["bal_weight"]

In [20]:
events_train_val, events_test, _, _ = train_test_split(
    events, targets_weights, test_size=0.2, random_state=42
)

In [16]:
def test_0_denom(events_test_0):
    events_test_0 = events_test_dfs[0]

    pn_c = events_test_0.ak4JetOutsidebtagPNetProbc + events_test_0.ak4JetOutsidebtagPNetProbcc
    pn_uds = events_test_0.ak4JetOutsidebtagPNetProbuds
    pn_g = events_test_0.ak4JetOutsidebtagPNetProbg

    return np.any((pn_c + pn_uds + pn_g) == 0)

In [ ]:
np.isnan(X_train.loc[:, "j3_btag"]).any()

## Load model

In [21]:
# Initialize the BDT model
bdt_model = XGBClassifier(
    n_estimators=196, max_depth=17, learning_rate=0.1, subsample=0.5, random_state=42
)

In [ ]:
print(np.sum(w_train))

In [ ]:
# Train the model
bdt_model.fit(X_train, y_train, sample_weight=w_train)

## Validate model

In [ ]:
y_val_scores = bdt_model.predict_proba(X_val)[:, 1]

fpr, tpr, thresholds = roc_curve(y_val, y_val_scores, sample_weight=w_val)
roc_auc = auc(fpr, tpr)

# histogram for y scores signal & background
# rank by importance

# number of b-jets that can be identified in resolved
# another feather cos(theta_star)

# Plottting
plt.figure()
plt.plot(tpr, fpr, color="darkorange", lw=2, label=f"ROC curve (area = {roc_auc:.2f})")
plt.xlabel("Signal (HH)")
plt.ylabel("Background (QCD&ttbar)")
plt.title("ROC")
plt.legend(loc="upper left")
plt.yscale("log")
plt.show()

## Calculate Yields

yeild = sum of weights_passed

### Calculate efficiecy per bin

In [25]:
hh_mass_bins = np.arange(400, 1601, 400)

In [26]:
threshold = 0.5

In [27]:
hh4b_scores = bdt_model.predict_proba(X_test.loc["hh4b"])[:, 1]
qcd_scores = bdt_model.predict_proba(X_test.loc["qcd"])[:, 1]
ttbar_scores = bdt_model.predict_proba(X_test.loc["ttbar"])[:, 1]

In [78]:
thresholds = [0.5, 0.7, 0.8, 0.9, 0.95, 0.99]
signal_yields = []
qcd_yields = []
ttbar_yields = []
bg_yields = []

for threshold in thresholds:
    signal_yield = events_test.loc["hh4b"][hh4b_scores > threshold]["weight_nonorm"].sum()
    qcd_yield = events_test.loc["qcd"][qcd_scores > threshold]["weight_nonorm"].sum()
    ttbar_yield = events_test.loc["ttbar"][ttbar_scores > threshold]["weight_nonorm"].sum()
    bg_yield = qcd_yield + ttbar_yield

    signal_yields.append(signal_yield)
    qcd_yields.append(qcd_yield)
    ttbar_yields.append(ttbar_yield)
    bg_yields.append(bg_yield)

In [ ]:
ttbar_yields

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(thresholds, signal_yields, label="signal")
ax.plot(thresholds, qcd_yields, label="qcd")
ax.plot(thresholds, ttbar_yields, label="ttbar")
ax.plot(thresholds, bg_yields, label="bg")
ax.legend()
ax.set(title="Yields vs. BDT Score Cut", xlabel="BDT Score Cut", ylabel="Yields")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(thresholds, signal_yields / np.sqrt(bg_yields))
ax.set(
    title="Signal/Sqrt(Background_Yield) vs. BDT Score Cut", xlabel="BDT Score Cut", ylabel="Ratio"
)
plt.show()

In [32]:
def get_HH_mass(events):
    fatjets = make_vector(events, "ak8FatJet")  # fatjets sorted by xbb
    h1 = fatjets[:, 0]

    jets_outside = make_vector(events, "ak4JetOutside")  # sorted by b-score
    j3 = jets_outside[:, 0]
    j4 = jets_outside[:, 1]
    h2 = j3 + j4

    hh = h1 + h2
    return hh.mass

In [33]:
hh_mass = get_HH_mass(events_test)

In [ ]:
bin_indices = np.digitize(hh_mass, hh_mass_bins)
print(np.min(bin_indices), np.max(bin_indices))

In [35]:
events_test["hh_mass"] = hh_mass

In [36]:
dtypes = events_test.dtypes

In [37]:
events_test_list_bins = [[], [], [], [], []]
for index, event in zip(bin_indices, events_test.iterrows()):
    events_test_list_bins[index].append(event[1])

In [38]:
events_test_dfs = [[], [], [], [], []]
for i, events_test_list_per_bin in enumerate(events_test_list_bins):
    events_test_dfs[i] = pd.concat(events_test_list_per_bin, axis=1).transpose().astype(dtypes)

In [39]:
def calc_eff_bmr(events, bdt, threshold):
    # eff = closest tpr

    # get x and y
    x = get_x_from_events(events)
    yw = get_yw_from_events(events)
    y = yw["target"]
    w = yw["weight"]

    # get predicted scores
    y_scores = bdt.predict_proba(x)[:, 1]
    fpr, tpr, thresholds = roc_curve(y, y_scores, sample_weight=w)
    nearest_threshold_index = np.abs(thresholds - threshold).argmin()

    eff = tpr[nearest_threshold_index]
    bmr = fpr[nearest_threshold_index]
    return eff, bmr

In [ ]:
effs_per_bins = []
bmrs_per_bins = []
for events_df_per_bin in events_test_dfs:
    eff, bmr = calc_eff_bmr(events_df_per_bin, bdt_model, threshold)
    effs_per_bins.append(eff)
    bmrs_per_bins.append(bmr)

In [ ]:
events_test_dfs[0]["weight_nonorm"]

## Make BDT score cuts

In [42]:
def BDT_classify(events, BDT_scores, cut):
    signal_idx = BDT_scores >= cut
    bg_idx = ~signal_idx

    signals = events[signal_idx]
    backgrounds = events[bg_idx]
    return signals, backgrounds

In [43]:
threshold = 0.95

In [44]:
BDTsignals_hh4b, _ = BDT_classify(events_test.loc["hh4b"], hh4b_scores, threshold)
BDTsignals_qcd, _ = BDT_classify(events_test.loc["qcd"], qcd_scores, threshold)
BDTsignals_ttbar, _ = BDT_classify(events_test.loc["ttbar"], ttbar_scores, threshold)

In [45]:
# Get BDT scores
hh4b_scores = bdt_model.predict_proba(X_test.loc["hh4b"])[:, 1]
qcd_scores = bdt_model.predict_proba(X_test.loc["qcd"])[:, 1]
ttbar_scores = bdt_model.predict_proba(X_test.loc["ttbar"])[:, 1]

BDT_scores = [hh4b_scores, qcd_scores, ttbar_scores]

## Construct analysis pipeline

Events -> BDT -> Signal/Bg

Plot signals with gen categories

In [46]:
# Get BDT scores
hh4b_scores = bdt_model.predict_proba(X_test.loc["hh4b"])[:, 1]
qcd_scores = bdt_model.predict_proba(X_test.loc["qcd"])[:, 1]
ttbar_scores = bdt_model.predict_proba(X_test.loc["ttbar"])[:, 1]

BDT_scores = [hh4b_scores, qcd_scores, ttbar_scores]

In [ ]:
plt.figure()
plt.hist(
    hh4b_scores,
    weights=events_test.loc["hh4b"]["weight"],
    bins=40,
    histtype="step",
    linewidth=1.5,
    color="darkblue",
)
plt.hist(
    qcd_scores,
    weights=events_test.loc["qcd"]["weight"],
    bins=40,
    histtype="step",
    linewidth=1.5,
    color="red",
)
plt.hist(
    ttbar_scores,
    weights=events_test.loc["ttbar"]["weight"],
    bins=40,
    histtype="step",
    linewidth=1.5,
    color="darkgreen",
)
plt.legend(["hh4b", "qcd", "ttbar"])
plt.ylabel("Frequency")
plt.xlabel("Prediciton Score")
plt.title("Model Predictions Histogram")
plt.yscale("log")
plt.show()

In [48]:
# Categorize with a threshold

In [49]:
def BDT_classify(events, BDT_scores, cut):
    signal_idx = BDT_scores >= cut
    bg_idx = ~signal_idx

    signals = events[signal_idx]
    backgrounds = events[bg_idx]
    return signals, backgrounds

In [50]:
## helper functions

In [51]:
def get_H1_mass(events):
    fatjets = make_vector(events, "ak8FatJet")  # fatjets sorted by xbb
    h1 = fatjets[:, 0]
    return h1.mass

In [52]:
def get_H2_mass(events):
    jets_outside = make_vector(events, "ak4JetOutside")  # sorted by b-score
    j3 = jets_outside[:, 0]
    j4 = jets_outside[:, 1]
    h2 = j3 + j4
    return h2.mass

In [53]:
def get_HH_mass(events):
    fatjets = make_vector(events, "ak8FatJet")  # fatjets sorted by xbb
    h1 = fatjets[:, 0]

    jets_outside = make_vector(events, "ak4JetOutside")  # sorted by b-score
    j3 = jets_outside[:, 0]
    j4 = jets_outside[:, 1]
    h2 = j3 + j4

    hh = h1 + h2
    return hh.mass

In [84]:
def analyze_SemiBoosted_BDT_output(events_test, BDT_scores, threshold, tag):
    hh4b_scores, qcd_scores, ttbar_scores = BDT_scores

    hh4b_weights = events_test.loc["hh4b"][hh4b_scores > threshold]["weight"]
    qcd_weights = events_test.loc["qcd"][qcd_scores > threshold]["weight"]
    ttbar_weights = events_test.loc["ttbar"][ttbar_scores > threshold]["weight"]

    BDTsignals_hh4b, _ = BDT_classify(events_test.loc["hh4b"], hh4b_scores, threshold)
    BDTsignals_qcd, _ = BDT_classify(events_test.loc["qcd"], qcd_scores, threshold)
    BDTsignals_ttbar, _ = BDT_classify(events_test.loc["ttbar"], ttbar_scores, threshold)

    # H1 mass
    h1_mass_hh4b = get_H1_mass(BDTsignals_hh4b)
    h1_mass_qcd = get_H1_mass(BDTsignals_qcd)
    h1_mass_ttbar = get_H1_mass(BDTsignals_ttbar)

    plt.figure()
    plt.hist(
        h1_mass_hh4b,
        bins=np.linspace(0, 250, 50),
        weights=hh4b_weights,
        histtype="step",
        linewidth=1.5,
        color="darkblue",
    )
    plt.hist(
        h1_mass_qcd,
        bins=np.linspace(0, 250, 50),
        weights=qcd_weights,
        histtype="step",
        linewidth=1.5,
        color="red",
    )
    plt.hist(
        h1_mass_ttbar,
        bins=np.linspace(0, 250, 50),
        weights=ttbar_weights,
        histtype="step",
        linewidth=1.5,
        color="darkgreen",
    )
    plt.legend(["hh4b", "qcd", "ttbar"])
    plt.ylabel("Frequency")
    plt.xlabel("H1 Mass (GeV)")
    plt.title(f"BDT SemiBoosted Signal's H1 Mass {tag}")
    plt.savefig(f"BDT_signal_H1_mass_{tag}.png")

    # H2 mass
    h2_mass_hh4b = get_H2_mass(BDTsignals_hh4b)
    h2_mass_qcd = get_H2_mass(BDTsignals_qcd)
    # h2_mass_ttbar = get_H2_mass(BDTsignals_ttbar)

    plt.figure()
    plt.hist(
        h2_mass_hh4b,
        bins=np.linspace(0, 250, 50),  # weights=hh4b_weights,
        histtype="step",
        linewidth=1.5,
        color="darkblue",
    )
    plt.hist(
        h2_mass_qcd,
        bins=np.linspace(0, 250, 50),  # weights=qcd_weights,
        histtype="step",
        linewidth=1.5,
        color="red",
    )
    # plt.hist(h2_mass_ttbar, bins=np.linspace(0, 250, 50), weights=ttbar_weights, histtype="step", linewidth=1.5, color="darkgreen")
    plt.legend(["hh4b", "qcd", "ttbar"])
    plt.ylabel("Frequency")
    plt.xlabel("H2 Mass (GeV)")
    plt.title(f"BDT SemiBoosted Signal's H2 Mass {tag}")
    plt.savefig(f"BDT_signal_H2_mass_{tag}.png")

    # HH mass
    hh_mass_hh4b = get_HH_mass(BDTsignals_hh4b)
    hh_mass_qcd = get_HH_mass(BDTsignals_qcd)
    # hh_mass_ttbar = get_HH_mass(BDTsignals_ttbar)

    plt.figure()
    plt.hist(
        hh_mass_hh4b,
        bins=np.linspace(0, 1800, 50),  # weights=hh4b_weights,
        histtype="step",
        linewidth=1.5,
        color="darkblue",
    )
    plt.hist(
        hh_mass_qcd,
        bins=np.linspace(0, 1800, 50),  # weights=qcd_weights,
        histtype="step",
        linewidth=1.5,
        color="red",
    )
    # plt.hist(hh_mass_ttbar, bins=np.linspace(0, 1800, 50), weights=ttbar_weights, histtype="step", linewidth=1.5, color="darkgreen")
    plt.legend(["hh4b", "qcd", "ttbar"])
    plt.ylabel("Frequency")
    plt.xlabel("HH Mass (GeV)")
    plt.title(f"BDT SemiBoosted Signal's HH mass {tag}")
    plt.savefig(f"BDT_signal_HH_mass_{tag}.png")

In [ ]:
analyze_SemiBoosted_BDT_output(events_test, BDT_scores, 0.99, "0.99")